In [1]:
# first make sure you've created a virtual environment and installed the requirements.txt
# ( python -m venv venv && source venv/bin/activate && pip install -r requirements.txt )

In [2]:
# now let's create a new proto-file, this defines the schema of the messages we want to send.
import os

f = open("car.proto", "x")

#fill it with your messages, in our case 2 mesages Car (message with one car) & Cars (message with multiple cars).
f.write("""
syntax = "proto3";

// the numbers are the ORDER the values will be sent using this protocoll buffer, must specified!
message Car {
    string brand = 1;
    string model = 2;
    int32 manufacturing_year = 3;
}

// A new message built for when we want to send multiple cars and not just one
message Cars {
    // reapeated= multiple values type
    // Car      = the message we want to repeat
    // cars     = the name of the variable in this message
    // 1        = the order of the variable in this message
    repeated Car cars = 1;
}
""")
        
f.close()

In [3]:
# then we will use this file as input to the protobuf compiler "protoc" (installed from grpc_tools)
# to generate a python file that we will import as a moduel and use to construct our messages

! python -m grpc_tools.protoc --proto_path=. --python_out=. ./car.proto

In [4]:
# Looking at the generated file 'car_pb2.py' we see no classes but only descriptors
# Python is a exception where other languanges get classes Pythons are generated in run time
# https://protobuf.dev/getting-started/pythontutorial/#protobuf-api :
# "
# Unlike when you generate Java and C++ protocol buffer code, the Python protocol buffer compiler doesn’t generate your data access code for you directly.
# Instead (as you’ll see if you look at your_file.py) it generates special descriptors for all your messages, enums, and fields,
# and some mysteriously empty classes, one for each message type
# "

In [5]:
# now we'll import the pythonfile car_pb2 and construct our messages
import car_pb2

car1 = car_pb2.Car()
car1.brand = "Toyota"
car1.model = "Camry"
car1.manufacturing_year = 2022

car2 = car_pb2.Car()
car2.brand = "Saab" # :)
car2.model = "900"
car2.manufacturing_year = 1987

cars = car_pb2.Cars()
cars.cars.extend([car1, car2])


In [11]:
# let's write our messages into two files, as a binary and as a json to compare the sizes of the data.

with open("messages.pb", "wb") as f:
    # our protobuf messages in it's binary output 
    f.write(cars.SerializeToString())

with open("messages.json", "x") as f:
    # our protobuf messages as string resembling json
    f.write(str(cars))

In [12]:
#then let' use a CLI command to list all files and see their size

! ls -lh

# WOW 136B vs 36B! that's a quater the size!!

total 64
-rw-r--r--  1 valdemarfinta  staff   1.6K Jun 12 16:05 README.md
drwxr-xr-x  3 valdemarfinta  staff    96B Jun 12 16:06 __pycache__
-rw-r--r--  1 valdemarfinta  staff   543B Jun 12 16:05 car.proto
-rw-r--r--  1 valdemarfinta  staff   1.1K Jun 12 16:05 car_pb2.py
-rw-r--r--  1 valdemarfinta  staff   6.6K Jun 12 16:08 main.ipynb
-rw-r--r--  1 valdemarfinta  staff   138B Jun 12 16:09 messages.json
-rw-r--r--  1 valdemarfinta  staff    36B Jun 12 16:09 messages.pb
-rw-r--r--  1 valdemarfinta  staff    36B Jun 12 16:08 requirements.txt
drwxr-xr-x  8 valdemarfinta  staff   256B Jun 12 16:01 venv
